In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers==4.3.3
!pip install stop_words
!pip install symspellpy
!pip install language_detector 
!pip install cached_property
!pip install sentencepiece
!pip install config
!pip install umap
!pip install sentence-transformers==0.4.1
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 7.3 MB/s 
     |████████████████████████████████| 880 kB 77.7 MB/s 
     |████████████████████████████████| 3.3 MB 72.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=3d8815def695a3b18d83fe76a5262d6fd0c529fb7b8ada377e57bc237e189e15
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=f561349eb2eb1735887137a6ed136f06cc26b0ee556a6af3fc9ce94213cc3524
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words
Looking in indexes: https://pypi.

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
import tensorflow, sentence_transformers, transformers
tensorflow.__version__, sentence_transformers.__version__, transformers.__version__

('2.9.2', '0.4.1', '4.3.3')

In [5]:
root_path = "/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2"
import sys, os
import config

config.root_path = os.path.abspath(root_path)
sys.path.insert(0, config.root_path)

In [6]:
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from src.encoders.context_encoder_ldabert_2 import ContextEncoderComplex
from src.dataset.ldabert_2 import LDABERT2Dataset

from tensorflow.python import keras
import toml
import json
import pandas as pd
import numpy as np
from utils.experiments import get_experiments_json, get_experiments, save_results

Downloading:   0%|          | 0.00/630 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/409 [00:00<?, ?B/s]

## Experiment

In [7]:
experiments_config = get_experiments_json('ldabert2_complex_test')
experiments_config_df = pd.DataFrame.from_dict(experiments_config)
experiments_config_df

,bert_type,dataset_type,final_dropout,dense_neurons,max_sentence_length,lstm_size,lstm_dropout_percentage,cnn_filters,cnn_kernel_size,pool_size,gamma,pct_data,augment_pct,bert_trainable,epochs
0,ldabert,clinical,0.25,256,128,256,0.25,8,3,2,15,1,1,False,200
1,ldabert,wiki,0.25,256,128,256,0.25,8,3,2,15,1,1,False,200
2,ldabert,fiction,0.25,256,128,256,0.25,8,3,2,15,1,1,False,200


In [8]:
experiments_config_df.to_csv(r'../models/experiment.csv', header=None, index=None, sep=' ', mode='a')

In [9]:
def get_random_hash(k):
  import random, string
  x = ''.join(random.choices(string.ascii_letters + string.digits, k=k))
  return x

In [ ]:
dataset = LDABERT2Dataset(dataset_type="wiki",
                        pct_data=0.001,
                      max_seq_length=128,
                        max_segment_length=5,
                        augment_pct=0.001,
                        split="train",
                        artificial_segments=False)

sentences, tokenized_sentences, labels = dataset.process()

vectors_filename = '{}_{}_artificial-segments-{}.pkl'.format(dataset.pct_data, dataset.augment_pct, dataset.artificial_segments)

saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.get_saved_vectors("train", dataset.dataset_type, vectors_filename)

if len(saved_vectors) == 0:
    saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.create_vectors("train", dataset.dataset_type, vectors_filename)

left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(saved_tokenized_sentences)
lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(saved_vectors)

artificial segments False
artificial segments False


In [ ]:
for experiment in experiments_config[2:3]: # fiction
    dataset_type = experiment['dataset_type']
    final_dropout = experiment['final_dropout']
    dense_neurons = experiment['dense_neurons']
    lstm_size = experiment['lstm_size']
    lstm_dropout_percentage = experiment['lstm_dropout_percentage']
    cnn_filters = experiment['cnn_filters']
    cnn_kernel_size = experiment['cnn_kernel_size']
    pool_size = experiment['pool_size']
    
    pct_data = experiment['pct_data']
    augment_pct = experiment['augment_pct']
    gamma = experiment['gamma']
    max_sentence_length = experiment['max_sentence_length']
    bert_trainable = experiment['bert_trainable']
    epochs = experiment['epochs']
    BATCH_SIZE = 8
    print("params:", experiment)
    random_hash = get_random_hash(5)
    experiment['epochs'] = epochs

    model = ContextEncoderComplex(final_dropout=final_dropout,
                            dense_neurons=dense_neurons,
                                  lstm_size=lstm_size,
                              lstm_dropout_percentage=lstm_dropout_percentage,
                              cnn_filters=cnn_filters,
                              cnn_kernel_size=cnn_kernel_size,
                              pool_size=pool_size,
                          gamma=gamma,
                          max_sentence_length=max_sentence_length,
                            bert_trainable=bert_trainable)
    
    model([
              left_input, mid_input, right_input, 
              lda_left_input, lda_mid_input, lda_right_input
          ])
    
    model.summary()

params: {'bert_type': 'ldabert', 'dataset_type': 'wiki', 'final_dropout': 0.25, 'dense_neurons': 256, 'max_sentence_length': 128, 'lstm_size': 256, 'lstm_dropout_percentage': 0.25, 'cnn_filters': 8, 'cnn_kernel_size': 3, 'pool_size': 2, 'gamma': 15, 'pct_data': 1, 'augment_pct': 1, 'bert_trainable': False, 'epochs': 200}


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


sbert_output TFBaseModelOutputWithPooling(last_hidden_state=<tf.Tensor: shape=(10, 33, 768), dtype=float32, numpy=
array([[[-0.29700822, -0.05949531, -0.2801308 , ...,  0.0665898 ,
          0.9728082 ,  0.02158672],
        [-0.24573436,  0.12755036, -0.14035524, ..., -0.0313878 ,
          1.0066551 , -0.05797172],
        [-0.20297782, -0.24609604, -0.32038602, ...,  0.13327792,
          0.7823258 , -0.1768093 ],
        ...,
        [-0.3222113 , -0.24850117, -0.25563157, ...,  0.21296376,
          0.6455367 , -0.17500459],
        [-0.24445531, -0.30231795, -0.13598728, ...,  0.02288361,
          0.6510857 , -0.11703973],
        [-0.5152119 , -0.09295473, -0.11663004, ...,  0.18098962,
          0.62915516, -0.05214427]],

       [[-0.27817464,  0.89690495,  0.1993424 , ..., -0.14764416,
          0.01439103,  0.47739184],
        [-0.55831647,  1.0294958 ,  0.34995764, ...,  0.04323666,
          0.12507579, -0.3097619 ],
        [ 0.22209328,  1.2667737 ,  0.3258406 , ..., -

In [ ]:
len(dataset.sentences)

7

In [10]:
for experiment in experiments_config[0:3]: # all
    dataset_type = experiment['dataset_type']
    final_dropout = experiment['final_dropout']
    dense_neurons = experiment['dense_neurons']
    lstm_size = experiment['lstm_size']
    lstm_dropout_percentage = experiment['lstm_dropout_percentage']
    cnn_filters = experiment['cnn_filters']
    cnn_kernel_size = experiment['cnn_kernel_size']
    pool_size = experiment['pool_size']
    
    pct_data = experiment['pct_data']
    augment_pct = experiment['augment_pct']
    gamma = experiment['gamma']
    max_sentence_length = experiment['max_sentence_length']
    bert_trainable = experiment['bert_trainable']
    epochs = experiment['epochs']
    BATCH_SIZE = 64
    print("params:", experiment)
    random_hash = get_random_hash(5)
    experiment['epochs'] = epochs
    
    # init model
    print("initializing model...")
    model = ContextEncoderComplex(final_dropout=final_dropout,
                            dense_neurons=dense_neurons,
                                  lstm_size=lstm_size,
                             lstm_dropout_percentage=lstm_dropout_percentage,
                             cnn_filters=cnn_filters,
                             cnn_kernel_size=cnn_kernel_size,
                             pool_size=pool_size,
                          gamma=gamma,
                          max_sentence_length=max_sentence_length,
                           bert_trainable=bert_trainable)
    
    # print("number of params: ", sum([np.prod(keras.get_value(w).shape) for w in model.trainable_weights]))
    
    # init dataset
    print("initializing dataset...")
    dataset = LDABERT2Dataset(dataset_type=dataset_type,
                            pct_data=pct_data,
                          max_seq_length=max_sentence_length,
                            max_segment_length=5,
                            augment_pct=augment_pct,
                            split="train",
                        artificial_segments=True)
    print("num samples", dataset.num_samples)

    # process dataset
    print("processing dataset...")
    sentences, tokenized_sentences, labels = dataset.process()

    vectors_filename = '{}_{}_artificial-segments-{}.pkl'.format(dataset.pct_data, dataset.augment_pct, dataset.artificial_segments)

    saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.get_saved_vectors("train", dataset.dataset_type, vectors_filename)

    if len(saved_vectors) == 0:
        saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.create_vectors("train", dataset.dataset_type, vectors_filename)

    left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(saved_tokenized_sentences)
    lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(saved_vectors)

    # get class weight
    neg, pos = np.bincount(labels.flatten())
    initial_bias = np.log([pos/neg])
    
    total=len(labels)
    weight_for_0 = (1 / neg)*(total)/2.0 
    weight_for_1 = (1 / pos)*(total)/2.0

    class_weight = {0: weight_for_0, 1: weight_for_1}
    print("class weight", class_weight)
    
    # create checkpoint path
    checkpoint_filepath = '{}/models/LDABERT2/complex/{}-{}-{}-pct-{}-aug_{}/finetune-{}/checkpoint'.format(
                            config.root_path,
                            dataset.dataset_type,                    
                            len(saved_sentences), 
                            dataset.pct_data,
                            dataset.augment_pct,
                            random_hash,
                            bert_trainable)
    
    # # UNCOMMENT IF YOU WANT TO CONTINUE TRAINING
    # random_hash = '6YPov'
    # checkpoint_filepath = '{}/models/LDABERT2/complex/{}-{}-{}-pct-{}-aug_{}/finetune-{}/checkpoint'.format(
    #                         config.root_path,
    #                         dataset.dataset_type,                    
    #                         len(saved_sentences), 
    #                         dataset.pct_data,
    #                         dataset.augment_pct,
    #                         random_hash,
    #                         bert_trainable)
    
    model_filepath = "/".join(checkpoint_filepath.split("/")[:-1]) + "/model.h5"
    print(checkpoint_filepath)
    
    # get callbacks ready.
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=False,
        monitor='val_accuracy',
        save_best_only=True,
        mode="auto",
        save_freq="epoch")

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy', 
        verbose=1,
        patience=10,
        mode='max',
        restore_best_weights=True)
    
    log_path = '{}/models/LDABERT2/complex/{}-{}-{}-pct-{}-aug_{}/finetune-{}/training.log'.format(
                            config.root_path,
                            dataset.dataset_type,                    
                            len(saved_sentences), 
                            dataset.pct_data,
                            dataset.augment_pct,
                            random_hash,
                            bert_trainable)
    
    csv_logger = tf.keras.callbacks.CSVLogger(log_path, separator=",", append=False)

    callbacks = [
    #     early_stopping,
        model_checkpoint_callback,
        csv_logger
    ]
    
    # compiling model
    print("compiling the model...")
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=[
                      keras.metrics.BinaryAccuracy(name='accuracy')
                  ])
    
    try:
        model.load_weights(checkpoint_filepath)
        print("model loaded.")
    except:
        print("No checkpoint available.")
    
    print("starting the training process...")
    # remove warnings
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
    history = model.fit([
                            left_input, mid_input, right_input, 
                            lda_left_input, lda_mid_input, lda_right_input
                        ], 
                        tf.convert_to_tensor(saved_labels), 
                        epochs=epochs,
                        validation_split=0.25,
                        batch_size=BATCH_SIZE,
                        verbose=1, 
                        class_weight=class_weight,
                        callbacks=callbacks)

    # assigning history to experiment object for saving.
    experiment["history"] = history.history
    experiment["hash"] = random_hash
    
    print("saving results...")
    save_results(experiment)

params: {'bert_type': 'ldabert', 'dataset_type': 'clinical', 'final_dropout': 0.25, 'dense_neurons': 256, 'max_sentence_length': 128, 'lstm_size': 256, 'lstm_dropout_percentage': 0.25, 'cnn_filters': 8, 'cnn_kernel_size': 3, 'pool_size': 2, 'gamma': 15, 'pct_data': 1, 'augment_pct': 1, 'bert_trainable': False, 'epochs': 200}
initializing model...


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


initializing dataset...
artificial segments True
artificial segments True
num samples 39968
processing dataset...
something went wrong [Errno 2] No such file or directory: '/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/data/lda_bert_2/generated_vectors/train/clinical/1_1_artificial-segments-True.pkl'
root path /content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2
Preprocessing raw texts ...
sentences length 39968
Preprocessing raw texts. Done!
lda sentences length 39968
Getting vector representations for LDA ...
Getting vector representations for LDA. Done!
saving vectors... 39968 39968 39968
class weight {0: 0.5438859102414065, 1: 6.196589147286822}
/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/models/LDABERT2/complex/clinical-39968-1-pct-1-aug_KBkZn/finetune-False/checkpoint
compiling the model...
No checkpoint available.
starting the training process...
Epoch 1/200
sbert_output TFBaseModelOutputWithPooling(last_hidden_state=<tf.Tensor 'context_encoder_com

469/469 [==============================] - 1131s 2s/step - loss: 0.5542 - accuracy: 0.9300 - val_loss: 0.5048 - val_accuracy: 0.9300
Epoch 2/200
469/469 [==============================] - 1017s 2s/step - loss: 0.5539 - accuracy: 0.9255 - val_loss: 0.5983 - val_accuracy: 0.9241
Epoch 3/200
469/469 [==============================] - 1017s 2s/step - loss: 0.5528 - accuracy: 0.9231 - val_loss: 0.5940 - val_accuracy: 0.9224
Epoch 4/200
469/469 [==============================] - 1017s 2s/step - loss: 0.5515 - accuracy: 0.9219 - val_loss: 0.5596 - val_accuracy: 0.9216
Epoch 5/200
469/469 [==============================] - 1017s 2s/step - loss: 0.5516 - accuracy: 0.9214 - val_loss: 0.5710 - val_accuracy: 0.9211
Epoch 6/200
469/469 [==============================] - 1017s 2s/step - loss: 0.5494 - accuracy: 0.9207 - val_loss: 0.5503 - val_accuracy: 0.9203
Epoch 7/200
469/469 [==============================] - 1017s 2s/step - loss: 0.5496 - accuracy: 0.9202 - val_loss: 0.5783 - val_accuracy: 0.92

KeyboardInterrupt: ignored

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()